In [9]:
import folium
import json
import math
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point, MultiPolygon
import geopandas as gpd
from geopandas.plotting import plot_polygon_collection
import matplotlib.pyplot as plt


In [10]:
cities = pd.read_csv("data/cities.csv", sep=" ", names=["lon", "lat", "city"])
temperatures = pd.read_csv("data/temperature.txt", sep=" ", names=["lon", "temp", "days", "day"])
troops = pd.read_csv("data/troops.txt", sep=" ", names=["lon", "lat", "survivors", "direction", "division"])

In [11]:
df_troops = pd.DataFrame(troops)
geometry = [Point(xy) for xy in zip(df_troops.lon, df_troops.lat)]
crs = {'init': 'epsg:4326'}
gdf_troops = gpd.GeoDataFrame(troops, crs=crs, geometry=geometry)

In [12]:
gdf_troops

,lon,lat,survivors,direction,division,geometry
0,24.0,54.9,340000,A,1,POINT (24 54.9)
1,24.5,55.0,340000,A,1,POINT (24.5 55)
2,25.5,54.5,340000,A,1,POINT (25.5 54.5)
3,26.0,54.7,320000,A,1,POINT (26 54.7)
4,27.0,54.8,300000,A,1,POINT (27 54.8)
5,28.0,54.9,280000,A,1,POINT (28 54.9)
6,28.5,55.0,240000,A,1,POINT (28.5 55)
7,29.0,55.1,210000,A,1,POINT (29 55.1)
8,30.0,55.2,180000,A,1,POINT (30 55.2)
9,30.3,55.3,175000,A,1,POINT (30.3 55.3)


In [16]:
coords = []
for i, row in gdf_troops.iterrows():
    coords.append([row.geometry.y, row.geometry.x])


In [27]:
m = folium.Map(location=[54,31],
               tiles='OpenStreetMap',
               zoom_start=5,
               control_scale=True)


In [ ]:
gdf_troops['buffer'] = (gdf_survivors / 34000) *.03


In [57]:
poly_list = []

for i, row in gdf_troops.iterrows():
    x1, y1 = coords[i]
    x2, y2 = coords[i+1]


        if x2 - x1 != 0:
            m = (y2 - y1) / (x2 - x1)
            #negative inverse slope (perpindicular angle)
            m_inv = (-1 / m)
        else:
            m_inv = 0

            c = 1 / math.sqrt(1 + (m_inv**2))
            s = m_inv / math.sqrt(1 + (m_inv**2))

            gdf_survivors = gdf_troops['survivors']
            gdf_direction = gdf_troops['direction']
            buffer = (gdf_survivors[i] / 34000) *.03

            
            
        
            
            


        lat_point_list = [x1u, x1d, x2d, x2u]
        lon_point_list = [y1u, y1d, y2d, y2u]
        
        poly_list.append([lat_point_list, lon_point_list])
            
print(poly_list)

IndentationError: unexpected indent (<ipython-input-57-a0bb3aeacfbe>, line 8)

In [53]:
m = folium.Map(location=[54,31],
               tiles='OpenStreetMap',
               zoom_start=5,
               control_scale=True)

def draw_polygons(mapobj, gdf_troops):
    
    for i, row in gdf_troops.iterrows():
        
        if i == len(gdf_troops) - 1:
            pass
        
        else:
            
            x1, y1 = coords[i]
            x2, y2 = coords[i+1]

            if x2 - x1 != 0:
                #slope of line
                slope = (y2 - y1) / (x2 - x1)
                #negative inverse slope (perpindicular angle)
                slope_inv = (-1 / slope)
            else:
                slope_inv = 0

            #cosine, sine to find x, y coordinates of polygon corners
            c = 1 / math.sqrt(1 + (slope_inv**2))
            s = slope_inv / math.sqrt(1 + (slope_inv**2))

            #find width of polygon (magnitude of survivors)
            gdf_survivors = gdf_troops['survivors']
            gdf_direction = gdf_troops['direction']
            #abstract max survivors
            #move outside of function and make new column
            buffer = (gdf_survivors[i] / 34000) *.03

            x1u, y1u = (x1 + (buffer*c)), (y1 + (buffer*s))
            x1d, y1d = (x1 - (buffer*c)), (y1 - (buffer*s))
            x2u, y2u = (x2 + (buffer*c)), (y2 + (buffer*s))
            x2d, y2d = (x2 - (buffer*c)), (y2 - (buffer*s))

            lat_point_list = [(x1 + (buffer*c)), (x1 - (buffer*c)), (x2 - (buffer*c)), (x2 + (buffer*c))]
            lon_point_list = [(y1 + (buffer*s)), (y1 - (buffer*s)), (y2 - (buffer*s)), (y2 + (buffer*s))]     

            
            
            geometry = list(zip(lat_point_list, lon_point_list))



            folium.Polygon(geometry, fill_color = 'black').add_to(mapobj)
            print(geometry)





    return mapobj
    

In [54]:
draw_polygons(m, gdf_troops)

[(55.19417420270727, 23.941165159458546), (54.605825797292724, 24.058834840541454), (54.705825797292725, 24.558834840541454), (55.294174202707275, 24.441165159458546)]
[(55.268328157299976, 24.634164078649988), (54.731671842700024, 24.365835921350012), (54.231671842700024, 25.365835921350012), (54.768328157299976, 25.634164078649988)]
[(54.778543007265576, 25.38858279709377), (54.221456992734424, 25.61141720290623), (54.42145699273443, 26.11141720290623), (54.97854300726558, 25.88858279709377)]
[(54.980951677235765, 25.971904832276426), (54.41904832276424, 26.028095167723574), (54.519048322764235, 27.028095167723574), (55.08095167723576, 26.971904832276426)]
[(55.063392197408525, 26.97366078025915), (54.53660780259147, 27.02633921974085), (54.63660780259147, 28.02633921974085), (55.16339219740853, 27.97366078025915)]
[(55.142261108111875, 27.951547778377623), (54.65773889188812, 28.048452221622377), (54.757738891888124, 28.548452221622377), (55.242261108111876, 28.451547778377623)]
[(5

In [2]:

max.gdf_survivors
style_function = lambda x: {'fillColor': "#000000", opacity}
lat_point_list = [(x1 + (buffer*c)), (x1 - (buffer*c)), (x2 - (buffer*c)), (x2 + (buffer*c))]
            lon_point_list = [(y1 + (buffer*s)), (y1 - (buffer*s)), (y2 - (buffer*s)), (y2 + (buffer*s))]     

                                                                                            

AttributeError: 'builtin_function_or_method' object has no attribute 'gdf_survivors'

In [3]:
gdf_troops

NameError: name 'gdf_troops' is not defined

In [119]:
polygon = Polygon(crs=crs, geometry=['polygon_geom'], style_function = style_function)
style_function = lambda x: {'fillColor': "#F09B39" if x['direction'] =='A' else '#000000'}
>>> GeoJson(geojson, style_function=style_function

In [100]:
    fillColor = orb_colors[products[key]["relativeorbitnumber"]]["fillColor"]


fillColor = 

In [75]:
from geopandas.plotting import plot_polygon_collection

fig, ax = plt.subplots()
ax.set_aspect('equal')
plot_polygon_collection(ax, gdf['geometry'], gdf['colors'], plot_values=False)

In [76]:
gdf_troops['direction']

0     A
1     A
2     A
3     A
4     A
5     A
6     A
7     A
8     A
9     A
10    A
11    A
12    A
13    A
14    A
15    A
16    R
17    R
18    R
19    R
20    R
21    R
22    R
23    R
24    R
25    R
26    R
27    R
28    A
29    A
30    A
31    A
32    A
33    A
34    R
35    R
36    R
37    R
38    R
39    R
40    R
41    R
42    R
43    R
44    R
45    A
46    A
47    A
48    R
49    R
50    R
Name: direction, dtype: object